In [15]:
import pandas as pd

#filepath variable
csv_loc = "Desktop/MSU_Data_bootcamp/pandas_challenge/pymoli/purchase_data.csv"

#read csv and create data frame
purch_df = pd.read_csv(csv_loc)

#show the dataframe as it currently is
purch_df.head()
print(len(purch_df))

780


In [16]:
#variable players should contain the length of the column of user names, with only unique values
#this gives the total number of players

players = len(pd.unique(purch_df["SN"]))

print("The numeber of unique players who have made a purchase is: " + str(players))

The numeber of unique players who have made a purchase is: 576


In [17]:
#number of unique items
uniq_it = purch_df[["Item Name", "Price"]].copy()                   #dataframe for just items and prices .copy() is added  
                                                                    #to clear up the SettingWithCopyWarning
uniq_it.drop_duplicates(subset = "Item Name", inplace = True)       #This operation drops duplicate sales

uniq_it_num = len(uniq_it)

#average price
price = round(uniq_it["Price"].mean(), 2)                           # .mean() and .sum() are used to compute properties of
                                                                    # columns in DataFrame["column"] 
pr_disp = "$" + str(price)

#total revenue
tot_rev = purch_df["Price"].sum()

rev_d = "$" + str(tot_rev)

#number of purchase
purch_no = len(purch_df)

#basic stats dataframe: dataframe is created from a dictionary of dict = {"column": [val1, val2, ... valn]}

bsc_stat = {"Unique Items": [uniq_it_num], "Average Price": [pr_disp], "Number of Purchases": [purch_no], "Total Revenue": [rev_d]}

bsc_stat_df = pd.DataFrame(bsc_stat, columns =["Unique Items", "Average Price", "Number of Purchases", "Total Revenue"])

bsc_stat_df

,Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.04,780,$2379.77


In [21]:
#gender demographics
uniq_users = purch_df[["SN", "Age", "Gender"]].copy()         #copy data from purch_df into new dataframe
uniq_users.drop_duplicates(subset = "SN", inplace = True)     # drop duplicates to get unique user data
#gender demographics
uniq_users = purch_df[["SN", "Age", "Gender"]].copy()         #copy data from purch_df into new dataframe
uniq_users.drop_duplicates(subset = "SN", inplace = True)     # drop duplicates to get unique user data

g_count = uniq_users["Gender"].value_counts()                 # g_count is a series of indexed counts of genders

m_perc = str(round((g_count[0]/players)*100, 2)) + "%"   #these are calculations of the percents. 

f_perc = str(round((g_count[1]/players)*100, 2)) + "%"    # I might try to simplify this with a loop or function

o_perc = str(round((g_count[2]/players)*100, 2)) + "%"        # because it seems like there is redundancy here


gndr_stats = {
                "Gender": ["Male", "Female", "Other/Non-Disclosed"], 
                "Count": [g_count[0], g_count[1], g_count[2]], 
                "Percent of Players": [m_perc, f_perc, o_perc]}

gndr_df = pd.DataFrame(gndr_stats, columns = ["Gender", "Count", "Percent of Players"])

gndr_df


,Gender,Count,Percent of Players
0,Male,484,84.03%
1,Female,81,14.06%
2,Other/Non-Disclosed,11,1.91%


In [22]:
#groupby gender for the original dataframe

#first I'm making a copy dataframe with the revenue data 
gen_purch_df = purch_df[["Gender", "Price"]].copy()

#group by gender and display average price per gender per purchase
ave_pur_price = gen_purch_df.groupby("Gender").mean()

ave_pur_price = ave_pur_price.rename(columns = {"Price":"Ave Purchase Price"})

#group by gender and display the sum of purchases per gender
gen_tot_rev = gen_purch_df.groupby("Gender").sum()

#groupby gender and count the purchases for each gender
gen_purch_ct = gen_purch_df.groupby("Gender").count()

gen_purch_ct = gen_purch_ct.rename(columns = {"Price":"Purchase Count"})

#these are variables to hold individual revenue totals by gender category

mtr = gen_tot_rev.loc["Male", "Price"]   

ftr = gen_tot_rev.loc["Female", "Price"]

otr = gen_tot_rev.loc["Other / Non-Disclosed", "Price"]

#this function is to format dollar amounts

def price(n):
    return "$" + str(round(n ,2))

#average spending for a member of each gender category

av_m_sp = price(mtr/g_count[0])

av_f_sp = price(ftr/g_count[1])

av_o_sp = price(otr/g_count[2])

#make dataframe for above calculations

av_sp = {"Gender": [ "Female", "Male", "Other / Non-Disclosed"],
        "Ave Total Purchase": [av_f_sp, av_m_sp, av_o_sp]}

av_sp_df = pd.DataFrame(av_sp, columns = ["Gender", "Ave Total Purchase"])

#rename the price column for the total revenue calculation
gen_tot_rev = gen_tot_rev.rename(columns = {"Price":"Total Revenue"})


#merging dataframes created by groupby operations into one dataframe
#this required 3 steps to do - it doesn't seem like you can merge multiple frames at once

m1 = pd.merge(gen_purch_ct, av_sp_df, on = "Gender")

m2 = pd.merge(ave_pur_price, gen_tot_rev, on = "Gender")

gender_stats = pd.merge(m1, m2, on = "Gender")

gender_stats

,Gender,Purchase Count,Ave Total Purchase,Ave Purchase Price,Total Revenue
0,Female,113,$4.47,3.203009,361.94
1,Male,652,$4.07,3.017853,1967.64
2,Other / Non-Disclosed,15,$4.56,3.346000,50.19


In [23]:
#in this cell I will bin purch_df then drop duplicates under a different name

ages = [5, 9, 14, 19, 24, 29, 34, 39, 45]

ag_bn = ["< 10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

purch_df["age_bin"] = pd.cut(purch_df["Age"], ages, labels = ag_bn, include_lowest = True)

#ab_df is my dataframe for age analysis. I'm dropping duplicates from it so I get a picture of users, not purchases
ab_df = purch_df[["SN", "age_bin"]].copy()
ab_df.drop_duplicates(subset = "SN", inplace = True)

ab_df = ab_df.groupby("age_bin").count()

ab_df = ab_df.rename(columns = {"SN":"Count"})

#I will use a list comprehension to calculate and format percentages

age_prcnt = [str(round((count/players)*100, 2)) + "%" for count in ab_df["Count"]]

#I'm now making that list into a dictionary so I can define a new dataframe to merge with ag_bn

age_prcnt_d = {"age_bin":ag_bn, "Percentages":age_prcnt}

age_prcnt_df = pd.DataFrame(age_prcnt_d, columns = ["age_bin", "Percentages"])

age_anl = pd.merge(ab_df, age_prcnt_df, on = "age_bin")

age_anl.set_index('age_bin')

age_anl


,age_bin,Count,Percentages
0,< 10,17,2.95%
1,10-14,22,3.82%
2,15-19,107,18.58%
3,20-24,258,44.79%
4,25-29,77,13.37%
5,30-34,52,9.03%
6,35-39,31,5.38%
7,40+,12,2.08%


In [48]:
#to start the I will use the purch_df dataframe to make a new frame for analysis, absent unneccesary data

pur_age = purch_df[["age_bin", "Price"]].copy()

#total revenue by age bin

rev_age_bin = pur_age.groupby("age_bin").sum()

#average purchase price by agebin

avpr_ab = pur_age.groupby("age_bin").mean()

#purchase count by age bin

ctpr_ab = pur_age.groupby("age_bin").count()

#this will calculate the average expendature per person and put it in a new data frame

a_t_p_p = rev_age_bin["Price"]/ab_df["Count"]

#I figured out that the operation above returned a series. I had to convert that to a dataframe to manipulate it
# as I desired. The code to do the conversion is in the line below. 

a_t_p_p = a_t_p_p.to_frame().reset_index()

#this is a section for renameing columns

rev_age_bin = rev_age_bin.rename(columns = {"Price":"Total Revenue"})

avpr_ab = avpr_ab.rename(columns = {"Price":"Average Purchase Price"})

ctpr_ab = ctpr_ab.rename(columns = {"Price":"Purchase Count"})

a_t_p_p = a_t_p_p.rename(columns = {0:"Average Revenue per Person"})

#This is for merging the results into a single dataframe

u1 = pd.merge(rev_age_bin, a_t_p_p, on = "age_bin")

u2 = pd.merge(ctpr_ab, avpr_ab, on = "age_bin")

full_age_an = pd.merge(u2, u1, on = "age_bin")

full_age_an


,age_bin,Purchase Count,Average Purchase Price,Total Revenue,Average Revenue per Person
0,< 10,23,3.353478,77.13,4.537059
1,10-14,28,2.956429,82.78,3.762727
2,15-19,136,3.035956,412.89,3.858785
3,20-24,365,3.052219,1114.06,4.318062
4,25-29,101,2.900990,293.00,3.805195
5,30-34,73,2.931507,214.00,4.115385
6,35-39,41,3.601707,147.67,4.763548
7,40+,13,2.941538,38.24,3.186667
